# Denavit Hartenberg Notation

Kevin Walchko
Created: 10 July 2017

---

DH is an attempt to standardize how we represent serial manipulators (i.e., robot arms). It is typically one of the first ways you learn. It is really easy (methodical) to do forward kinematics, but becomes more challenging when doing inverse kinematics.

## Objectives

- understand coordinate frames
- apply rotations and translations to objects in 3d space
- calculate DH forward kinematics for a serial link mechanism
- understand homogenous transformations
- understand Euler sequences

## References

- [Wikipedia modified DH](https://en.wikipedia.org/wiki/Denavit%E2%80%93Hartenberg_parameters)
- [Wikipedia euler angles](https://en.wikipedia.org/wiki/Euler_angles#Tait.E2.80.93Bryan_angles)
- [darpa robot challenge](https://www.youtube.com/watch?v=diaZFIUBMBQ)
- [darpa robot fails](https://youtu.be/wX0KagJ1du8)
- [Walking robot](https://vimeo.com/194676675)
- [Euler angles](https://www.learnopencv.com/rotation-matrix-to-euler-angles/)

## Setup

In [1]:
%matplotlib inline

In [2]:
from __future__ import print_function
from __future__ import division
import numpy as np

from IPython.display import HTML # need this for embedding a movie in an iframe

# Kinematics of Serial Manipulators

Robot arms and legs are hard to control (lots of math), but required for most robotic applications. Kinematics is the study of motion without regard to the forces which cause it. Kinematics of manipulators involves the study of the geometric and time based properties of the motion, and in particular how the various links move with respect to one another and with time.

**Pose:** The combination of position and orientation of a frame relative to a reference or inertial frame. Think [*Zoolander*](http://www.imdb.com/title/tt0196229/) on a fashion shoot, "strike a pose"

This lesson will talk about matrix and vector operations. A nice review of the various mathematical operators is [wikipedia](https://en.wikipedia.org/wiki/Matrix_multiplication). Take a look at how you multiply 2 matrices together (Matrix Product (two matrices)) and it will give you an idea of how it works. Ultimately we will use `numpy` to do these operations for us.

## Coordinate Frames

![](dh_pics/right_hand_axes.jpg)

We want to describe positions and orientations of bodies in space relative to a [reference](https://en.wikipedia.org/wiki/Frame_of_reference) (or in some cases an [inertial](https://en.wikipedia.org/wiki/Inertial_frame_of_reference)) frame.

## Manipulators

![](dh_pics/parallel.jpg)

![](dh_pics/serial.png)

There are 2 types of manipulators in robotics: serial and parallel. They are used in different application for manufacturing. 

| Type     | Pro                     | Con                        |
|----------|-------------------------|----------------------------|
| Serial   | easy forward kinematics | complex inverse kinematics |
| Parallel | easy inverse kinematics | complex forward kinematics |

For this class we are going to focus on serial manipulators. 

## Serial Manipulators

- We generally draw a simplified version of the manipulator and attach a “frame” to each rigid body link
    - The simplified version only has:
        - *Revolute joints:* joints that rotate
        - *Prismatic joints:* joints that move linearly (think telescoping)
    - By combining these in various combinations, we can make anything. For example, a spherical joint (ball-and-socket like your shoulder) is generally composed of 3 co-located (not physically real) rotational joints. This makes the math easier.
- The frames follow the serial manipulator's body
- All of our frames will be a righthand-coordinate system (RHS) ... sorry lefties
- There's some freedom in how we choose the frame's position and orientation relative to the body
- DH notation partially standardizes this process, however, there are classical DH parameters and modified DH parameters. We are using modified (see reading in Craig). In the end you end up with the same equations, however, some people (i.e., Craig) didn't like how the classical notation was written.

An example (sort of) is shown below. Notice the real robot is represented as a simpler drawing of little metal bars:

![](dh_pics/kuka-kr270.png)

The KR270 has 6 joints and is a standard industrial type robot for manufacturing. Notice again, the wrist, is represented as 2 revolute joints co-located (on top of each other) which is not realistic.

## Euler Angles

<img src="dh_pics/euler.png" width="400px">

Euler angles describe the moving frame and not the fixed (inertial) frame. They follow the sequence (see Craig eqn 2.70) will descibe the end-effector (or the aircraft pictured above) relative to the fixed frame:

- **Roll:** rotate about the x-axis
- **Pitch:** rotate about (the new) y-axis
- **Yaw:** rotate about (the new) y-axis

Please notice, each rotation is dependant on previous rotations. Also notice, you can tell which axis the rotation is around by looking for the **1** in the matrix.

\begin{eqnarray}
  R_x(\gamma) = \begin{bmatrix}
    1 & 0 & 0 \\
    0 & \cos(\gamma) & -\sin(\gamma) \\
    0 & \sin(\gamma) & \cos(\gamma)
  \end{bmatrix} \\
  R_y(\beta) = \begin{bmatrix}
    \cos(\beta) & 0 & \sin(sin(\beta) \\
    0 & 1 & 0 \\
    -\sin(\beta) & 0 & \cos(\beta)
  \end{bmatrix} \\
  R_z(\alpha) = \begin{bmatrix}
    \cos(\alpha) & -\sin(\alpha) & 0 \\
    \sin(\alpha) & \cos(\alpha) & 0 \\
    0 & 0 & 1
  \end{bmatrix} \\
  R^A_B = R_z(\alpha) R_y(\beta) R_x(\gamma)
\end{eqnarray}


Now, for this class, we are not going to use these here, but you will see them pop-up in later lessons which is why I mention them. A nice thing about these rotation matricies, is:

$$
R^B_A = (R^A_B)^T = (R^A_B)^{-1}
$$

This is because they are orthonormal (the dot product of the x, y, and z components is 0). This means a matrix inverse and matrix transpose produce the same results, which is good since matrix inverse is CPU intensive compared to a matrix transpose (shown below).

$$
R = \begin{bmatrix}
  1 & 2 & 3 \\
  4 & 5 & 6 \\
  7 & 8 & 9
\end{bmatrix} \\
R^T = \begin{bmatrix}
  1 & 4 & 7 \\
  2 & 5 & 8 \\
  3 & 6 & 9
\end{bmatrix}
$$

Notice a transpose just turns matrix columns into matrix rows ... no math!

## Position and Orientation

[](dh_pics/frame1.png)

<img src="dh_pics/frame1.png" width="400px">

Now the position/orientation, or pose, of {B} relative to {A} is:

$$
P^A = R^A_B P^B
$$

Now ultimately, we want to know the location of a point in {B} relative to an inertial frame (say the base of our robot arm) in {A}.

<img src="dh_pics/frame2.png" width="400px">

<img src="dh_pics/frame3.png" width="400px">

What the final equation means, is, the position of $P^B$ in frame A is equal to the offset between {A} and {B} (i.e., $P^A_B$) plus the change in orientation between {A} and {B} (i.e., $R^A_B P^B$). 

A compact representation of the translation and rotation is known as the **Homogeneous Transformation**. This allows us to cast the rotation and translation of the general transform in a single matrix form.

$$
T^A_B = \begin{bmatrix}
   & R^A_B & & P^A_B \\
  0 & 0 & 0 & 1
\end{bmatrix} \\
\begin{bmatrix}
  P^A \\
  1
\end{bmatrix} = 
T^A_B
\begin{bmatrix}
  P^B \\
  1
\end{bmatrix}
$$

This compact notation allows us to use `numpy` or `matlab` to write series of equations as matricies and do standard matrix operations on them. Now, as we attach frames to serial manipulator links, we will be able to combine these matricies together to calculate where the end effector is.

<img src="dh_pics/frame4.png" width="600px">

# Denavit-Hartenberg (DH)

Development of the DH forward kinematics for a robot. We will use the DH method to develop the symbolic equations and use python to simplify them.

![](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d8/DHParameter.png/609px-DHParameter.png)

## Process (Craig, section 3.4, pg 67)

Now using ideas from before (see Craig, eqns 3.4 and 3.6):

| | | |
|------------|:---------|:------------------------------------------------------|
| $a_i$      | length   | distance from $z_i$ to $z_{i+1}$ measured along $x_i$ |
| $d_i$      | offset   | distance from $x_{i-1}$ to $x_i$ along $z_i$          |
| $\alpha_i$ | twist    | angle from $z_i$ to $z_{i+1}$ measured about $x_i$    |
| $\theta_i$ | rotation | angle from $x_{i−1}$ to $x_i$ measured about $z_i$    |

Summary of steps:

1. Identify the joint axes and imagine (or draw) infinite lines along them. For steps 2 through 5 below, consider two of these neighboring lines (at axes i and i+1).
1. Identify the comon perpendicular betwen them,or point of intersection. At the point of intersection, or at the point where the comon perpendicular mets the ith axis, asign the link-frame origin.
1. Asign the $Z_i$ axis pointing along the ith joint axis.
1. A sign the axis pointing along the comon perpendicular, or, if the axes intersect, asign k1 to be normal to the plane containing the two axes.
1. Asign the y axis to complete a right-hand cordinate system ... honestly don't draw y-axis 
1. Asign {0} to match {1} when the first joint variable is zero. For {N}, chose an origin location and $X_N$ direction freely, but generaly so as to cause as many linkage parameters as possible to become zero.

Now, once you have the parameters, you can enter them into the followng matrix to get the relationship between from i and frame i+1. Note, that these are **not** euler angles, but rather:

1. A translation along z by d
1. Rotation about z by $\theta$
1. translation along x by a
1. Rotation about x by $\alpha$

$$
\begin{eqnarray}
T^{i-1}_i = R_x(\alpha_{i-1}) D_x(a_{i-1}) R_z(\theta_i) D_z(d_i) \\
T^{i-1}_i = \begin{bmatrix}
  \cos(\theta_i)                   & -\sin(\theta_i)                  & 0                   & a_{i-1} \\
  \sin(\theta_i)\cos(\alpha_{i-1}) & \cos(\theta_i)\cos(\alpha_{i-1}) & -\sin(\alpha_{i-1}) & -\sin(\alpha_{i-1})d_i \\
  \sin(\theta_i)\sin(\alpha_{i-1}) & \cos(\theta_i)\sin(\alpha_{i-1}) & \cos(\alpha_{i-1})  & \cos(\alpha_{i-1})d_i \\
  0                                & 0                                & 0                   & 1
\end{bmatrix}
\end{eqnarray}
$$

Then you can multiply these matricies together to get the transform from base frame {0} to end effector frame {3} by: $T^0_3 = T^0_1 T^1_2 T^2_3$

In [ ]:
HTML('<iframe src="https://player.vimeo.com/video/238147402" width="640" height="360" frameborder="0" webkitallowfullscreen mozallowfullscreen allowfullscreen></iframe><p><a href="https://vimeo.com/238147402">Denavit&ndash;Hartenberg parameters</a> from <a href="https://vimeo.com/user59907133">kevin</a> on <a href="https://vimeo.com">Vimeo</a>.</p>')

# KUKA KR270 Example

![](dh_pics/kuka-kr270.png)

Now following the steps above, we get (**note:** my frames are a little different than the picture):

| Link | $a_{i-1}$ | $\alpha_{i-1}$ | $d_i$ | $\theta_i$ |
|------|-----------|----------------|-------|------------|
| 1    | 0         | 0              | $d_1$ | $\theta_1$ |
| 2    | $a_1$     | 90             | 0     | $\theta_2$ |
| 3    | $a_2$     | 0              | 0     | $\theta_3$ |
| 4    | $a_3$     | 90             | $d_4$ | $\theta_4$ |
| 5    | 0         | 90             | $d_5$ | $\theta_5$ |
| 6    | 0         | -90            | 0     | $\theta_6$ |

## Simple 2D Example

Following the process for assigning frames to a manipulator, you get the following:

![](dh_pics/2d-ex.png)

Looking at the above frames, they are related by:

| Link | $a_{i-1}$ | $\alpha_{i-1}$ | $d_i$ | $\theta_i$ |
|------|-----------|----------------|-------|------------|
| 1    | 0     | 0          | 0     | $\theta_1$ |
| 2    | $a_1$ | 0          | 0     | $\theta_2$ |
| 3    | $a_2$ | 0          | 0     | 0          |

Now using Craig eqn 3.6, we can substitute these values in and get the relationship between the inertial frame and the end effector. However note, $\theta_i$ are variable parameters. Typically we would simplify these equations down leaving only the theta parameters. Let's use the python symbolic toolbox to generate the equations of motion.

In [3]:
# Let's grab some libraries to help us manipulate symbolic equations
import sympy
from sympy import symbols, sin, cos, pi, simplify

def makeT(a, alpha, d, theta):
    # create a modified DH homogenious matrix
    return np.array([
        [           cos(theta),           -sin(theta),           0,             a],
        [sin(theta)*cos(alpha), cos(theta)*cos(alpha), -sin(alpha), -d*sin(alpha)],
        [sin(theta)*sin(alpha), cos(theta)*sin(alpha),  cos(alpha),  d*cos(alpha)],
        [                    0,                     0,           0,             1]
    ])

def simplifyT(tt):
    """
    This goes through each element of a matrix and tries to simplify it.
    """
    for i, row in enumerate(tt):
        for j, col in enumerate(row):
            tt[i,j] = simplify(col)
    return tt

# def subs(tt, m):
#     """
#     This allows you to simplify the trigonomic mess that kinematics can
#     create and also substitute in some inputs in the process
#     """
#     for i, row in enumerate(tt):
#         for j, col in enumerate(row):
#             try:
#                 tt[i,j] = col.subs(m)
#             except:
#                 tt[i,j] = col
#     return tt

In [4]:
# make thetas (t) and link lengths (a) symbolics
t1, t2 = symbols('t1 t2')
a1, a2 = symbols('a1 a2')

# let's create our matrices
T1 = makeT(0, 0, 0, t1)
T2 = makeT(a1, 0, 0, t2)
T3 = makeT(a2, 0, 0, 0)

# T13 = T1 * T2 * T3
T13 = T1.dot(T2.dot(T3))

print('T1 = ', T1)
print('T2 = ', T2)
print('T3 = ', T3)
print('\nSo the combined homogenious matrix is:\n')
print('T13 = ', T13)

T1 =  [[cos(t1) -sin(t1) 0 0]
 [sin(t1) cos(t1) 0 0]
 [0 0 1 0]
 [0 0 0 1]]
T2 =  [[cos(t2) -sin(t2) 0 a1]
 [sin(t2) cos(t2) 0 0]
 [0 0 1 0]
 [0 0 0 1]]
T3 =  [[1 0 0 a2]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]]

So the combined homogenious matrix is:

T13 =  [[-sin(t1)*sin(t2) + cos(t1)*cos(t2) -sin(t1)*cos(t2) - sin(t2)*cos(t1) 0
  -a2*sin(t1)*sin(t2) + (a1 + a2*cos(t2))*cos(t1)]
 [sin(t1)*cos(t2) + sin(t2)*cos(t1) -sin(t1)*sin(t2) + cos(t1)*cos(t2) 0
  a2*sin(t2)*cos(t1) + (a1 + a2*cos(t2))*sin(t1)]
 [0 0 1 0]
 [0 0 0 1]]


So that looks a little messy with all of the sines and cosines ... let's use the python symbolic capabilities in `sympy` to help us reduce this a little and figure out where the end-effector is relative to the base (e.g. inertial frame).

In [5]:
ans = simplifyT(T13)
print(ans)
print('-'*25)
print('position x: {}'.format(ans[0,3]))
print('position y: {}'.format(ans[1,3]))

[[cos(t1 + t2) -sin(t1 + t2) 0 a1*cos(t1) + a2*cos(t1 + t2)]
 [sin(t1 + t2) cos(t1 + t2) 0 a1*sin(t1) + a2*sin(t1 + t2)]
 [0 0 1 0]
 [0 0 0 1]]
-------------------------
position x: a1*cos(t1) + a2*cos(t1 + t2)
position y: a1*sin(t1) + a2*sin(t1 + t2)


Later, we will derive the same 2 link manipulator a different way and come up with the same equations for position of the end-effector in the x, y plane. For simple manipulators, DH is overkill, however, for most real manipulators (remember the DARPA videos), it is useful to understand what is going on if you do robotics.

# Question

Practice makes perfect. Try to do the following from Craig so you will have success on the GR.

- Puma Figure 3.18, try to get the same DH parameters as in table 3-21.
- Yasukawa L-3, Figure 3.25, answers in table 3.26 

In [7]:
# craig puma
t1,t2,t3,t4,t5,t6 = symbols('t1 t2 t3 t4 t5 t6')
a2, a3, d3, d4 = symbols('a2 a3 d3 d4')

T1 = makeT(0,0,0,t1)
T2 = makeT(0,-pi/2,0,t2)
T3 = makeT(a2,0,d3,t3)
T4 = makeT(a3,-pi/2,d4,t4)
T5 = makeT(0,pi/2,0,t5)
T6 = makeT(0,-pi/2,0,t6)

# print(T45)
# print(T56)
# print(T45.dot(T56))

ans = np.eye(4)
for T in [T1, T2, T3, T4, T5, T6]:
    ans = ans.dot(T)

ans = simplifyT(ans)
print(ans)
print('-'*25)
print('position x: {}'.format(ans[0,3]))
print('position y: {}'.format(ans[1,3]))
print('position z: {}'.format(ans[2,3]))

[[ 1.0*((sin(t1)*sin(t4) + cos(t1)*cos(t4)*cos(t2 + t3))*cos(t5) - sin(t5)*sin(t2 + t3)*cos(t1))*cos(t6) + 1.0*(sin(t1)*cos(t4) - sin(t4)*cos(t1)*cos(t2 + t3))*sin(t6)
  1.0*(-(sin(t1)*sin(t4) + cos(t1)*cos(t4)*cos(t2 + t3))*cos(t5) + sin(t5)*sin(t2 + t3)*cos(t1))*sin(t6) + 1.0*(sin(t1)*cos(t4) - sin(t4)*cos(t1)*cos(t2 + t3))*cos(t6)
  -1.0*(sin(t1)*sin(t4) + cos(t1)*cos(t4)*cos(t2 + t3))*sin(t5) - 1.0*sin(t2 + t3)*cos(t1)*cos(t5)
  1.0*a2*cos(t1)*cos(t2) + 1.0*a3*cos(t1)*cos(t2 + t3) - 1.0*d3*sin(t1) - 1.0*d4*sin(t2 + t3)*cos(t1)]
 [ 1.0*((sin(t1)*cos(t4)*cos(t2 + t3) - sin(t4)*cos(t1))*cos(t5) - sin(t1)*sin(t5)*sin(t2 + t3))*cos(t6) - 1.0*(sin(t1)*sin(t4)*cos(t2 + t3) + cos(t1)*cos(t4))*sin(t6)
  1.0*((-sin(t1)*cos(t4)*cos(t2 + t3) + sin(t4)*cos(t1))*cos(t5) + sin(t1)*sin(t5)*sin(t2 + t3))*sin(t6) - 1.0*(sin(t1)*sin(t4)*cos(t2 + t3) + cos(t1)*cos(t4))*cos(t6)
  1.0*(-sin(t1)*cos(t4)*cos(t2 + t3) + sin(t4)*cos(t1))*sin(t5) - 1.0*sin(t1)*sin(t2 + t3)*cos(t5)
  1.0*a2*sin(t1)*cos(t2) + 

Looking at the position, this is the same position listed in Craig, eqn 3.14.


-----------

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.